In [1]:
# Installing all the necessary libraries we need
%pip install azure-keyvault-secrets
%pip install azure-identity
%pip install azure-keyvault-secrets
%pip install torch
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing the libraries to our workspace
import os
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import densenet201
from azureml.core import Workspace, Dataset, Experiment, Run

from __future__ import print_function, division
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import numpy as np
import time
import copy
import argparse
import mlflow
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.filedatalake import DataLakeServiceClient
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient

from azure.ai.ml import command
from azure.ai.ml import Input

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Authenticating acess to Azure Data Lake using Azure CLI
key_vault_name = 'serengetikeyvaultqezfaqk'
key_vault_url = f'https://{key_vault_name}.vault.azure.net/'
pool_conn_secret_name = 'DedicatedPool-odbc-ConnectionString'
adls_conn_secret_name = 'ADLS-ConnectionString'

credential = AzureCliCredential()
secret_client = SecretClient(vault_url=key_vault_url, credential = credential)

adls_connection_string = secret_client.get_secret(adls_conn_secret_name).value

In [ ]:
# # You can also authenticate using default Azure Credential, I could not as it gave me error 403
# # Handle to the workspace
# from azure.ai.ml import MLClient

# # Authentication package
# from azure.identity import DefaultAzureCredential

# credential = DefaultAzureCredential()

In [4]:
# Connect to the adls and check to see if you can be able to read the files
service_client = DataLakeServiceClient.from_connection_string(adls_connection_string)

file_system_name = "snapshot-serengeti"
file_path = "images/train"
file_system_client = service_client.get_file_system_client(file_system=file_system_name)
paths = file_system_client.get_paths(path=file_path)

for path in paths:
    print(path.name + "\n")

images/train/baboon

images/train/baboon/S3_P04_R11_IMAG0017.JPG

images/train/baboon/S3_P04_R11_IMAG0018.JPG

images/train/baboon/S3_P04_R11_IMAG0019.JPG

images/train/baboon/S7_I01_R1_IMAG0011.JPG

images/train/baboon/S7_I01_R1_IMAG0012.JPG

images/train/baboon/S7_I01_R1_IMAG0013.JPG

images/train/baboon/S7_I01_R1_IMAG0014.JPG

images/train/baboon/S7_I01_R1_IMAG0015.JPG

images/train/baboon/S7_I01_R1_IMAG0016.JPG

images/train/buffalo

images/train/buffalo/S10_I11_R2_IMAG0283.JPG

images/train/buffalo/S2_D05_R1_PICT2690.JPG

images/train/buffalo/S2_D05_R1_PICT2691.JPG

images/train/buffalo/S2_D05_R1_PICT2692.JPG

images/train/buffalo/S2_D05_R1_PICT2693.JPG

images/train/buffalo/S2_D05_R1_PICT2694.JPG

images/train/buffalo/S2_D05_R1_PICT2695.JPG

images/train/buffalo/S2_D05_R1_PICT2696.JPG

images/train/buffalo/S2_D05_R1_PICT2697.JPG

images/train/buffalo/S2_D05_R1_PICT2698.JPG

images/train/buffalo/S2_D05_R1_PICT2699.JPG

images/train/buffalo/S2_D05_R1_PICT2700.JPG

images/train/buff

In [6]:
# Get Azure Machine Learning workspace
ws = Workspace.from_config()

# Get Azure Machine Learning run context
run = Run.get_context()

In [23]:
# Get a handle to the workspace
# Handle to the workspace
from azure.ai.ml import MLClient
ml_client = MLClient(
    credential=credential,
    subscription_id="a1a27566-3e3c-42d7-a372-692095cd8521",
    resource_group_name="SerengetiDataLab",
    workspace_name="SerengetiAMLqezfaqkji2jnm",
)

In [7]:
# Get the data folder location
data_folder =  service_client.get_file_system_client(file_system=file_system_name)

# Define transforms for the data
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [21]:
# Create a job environment
# n Azure Machine Learning environment encapsulates the dependencies (such as software runtime and libraries) needed to run your machine learning training script on your compute resource
curated_env_name = "azureml://registries/azureml/environments/AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu/versions/11"


In [ ]:
# Python Script
def load_data(data_dir):
    """Load the train/val data."""
    train = service_client.get_file_system_client(file_system=file_system_name).get_paths(path="images/train/")
    val = service_client.get_file_system_client(file_system=file_system_name).get_paths(path="images/val/")    

    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        "train": transforms.Compose(
            [
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        ),
        "val": transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        ),
    }

    image_datasets = {
        x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
        for x in ["train", "val"]
    }
    dataloaders = {
        x: torch.utils.data.DataLoader(
            image_datasets[x], batch_size=4, shuffle=True, num_workers=4
        )
        for x in ["train", "val"]
    }
    dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "val"]}
    class_names = image_datasets["train"].classes

    return dataloaders, dataset_sizes, class_names


def train_model(model, criterion, optimizer, scheduler, num_epochs, data_dir):
    """Train the model."""

    # load training/validation data
    dataloaders, dataset_sizes, class_names = load_data(data_dir)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        print("-" * 10)

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

            # log the best val accuracy to AML run
            mlflow.log_metric("best_val_acc", np.float(best_acc))

        print()

    time_elapsed = time.time() - since
    print(
        "Training complete in {:.0f}m {:.0f}s".format(
            time_elapsed // 60, time_elapsed % 60
        )
    )
    print("Best val Acc: {:4f}".format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


def fine_tune_model(num_epochs, data_dir, learning_rate, momentum):
    """Load a pretrained model and reset the final fully connected layer."""

    # log the hyperparameter metrics to the AML run
    mlflow.log_metric("lr", np.float(learning_rate))
    mlflow.log_metric("momentum", np.float(momentum))

    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)  # only 2 classes to predict

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=momentum)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    model = train_model(
        model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs, data_dir
    )

    return model



def main():
    print("Torch version:", torch.__version__)

    # get command-line arguments
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--num_epochs", type=int, default=25, help="number of epochs to train"
    )
    parser.add_argument("--output_dir", type=str, help="output directory")
    parser.add_argument(
        "--learning_rate", type=float, default=0.001, help="learning rate"
    )
    parser.add_argument("--momentum", type=float, default=0.9, help="momentum")
    args = parser.parse_args()

    # Start Run
    mlflow.start_run()

    model = fine_tune_model(
        args.num_epochs, data_dir, args.learning_rate, args.momentum
    )
    os.makedirs(args.output_dir, exist_ok=True)
    torch.save(model, os.path.join(args.output_dir, "model.pt"))

    mlflow.end_run()


if __name__ == "__main__":
    main()

In [22]:
job = command(
    inputs=dict(
        num_epochs=30, learning_rate=0.001, momentum=0.9, output_dir="./outputs"
    ),
    compute="bethanyjep1",
    environment=curated_env_name,
    code="./src",  # location of source code
    command="python -c 'from Data.ipynb import main; train_model(num_epochs=${{inputs.num_epochs}}, output_dir=${{inputs.output_dir}})'",
    experiment_name="pytorch-serengeti",
    display_name="pytorch-serengeti-image",
)

In [24]:
# Submit the job
ml_client.jobs.create_or_update(job)

Experiment,Name,Type,Status,Details Page
pytorch-serengeti,jolly_leg_sybbh7g37y,command,Starting,Link to Azure Machine Learning studio


In [26]:
from azure.ai.ml.entities import AmlCompute
# Compute to be used in training
gpu_compute_taget = "bethanyjep1"


In [25]:
# tuning hyperparameters
from azure.ai.ml.sweep import Uniform

# we will reuse the command_job created before. we call it as a function so that we can apply inputs
job_for_sweep = job(
    learning_rate=Uniform(min_value=0.0005, max_value=0.005),
    momentum=Uniform(min_value=0.9, max_value=0.99),
)

In [27]:
from azure.ai.ml.sweep import BanditPolicy
# termination policy
sweep_job = job_for_sweep.sweep(
    compute="bethanyjep1",
    sampling_algorithm="random",
    primary_metric="best_val_acc",
    goal="Maximize",
    max_total_trials=8,
    max_concurrent_trials=4,
    early_termination_policy=BanditPolicy(
        slack_factor=0.15, evaluation_interval=1, delay_evaluation=10
    ),
)

In [28]:
# submiting your job with a sweep job running over your train job to tune your hyperparameters
returned_sweep_job = ml_client.create_or_update(sweep_job)

# stream the output and wait until the job is finished
ml_client.jobs.stream(returned_sweep_job.name)

# refresh the latest status of the job after streaming
returned_sweep_job = ml_client.jobs.get(name=returned_sweep_job.name)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


AzureCliCredential.get_token failed: --- Logging error ---
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/logging/__init__.py", line 1089, in emit
    self.flush()
  File "/anaconda/envs/azureml_py38/lib/python3.8/logging/__init__.py", line 1069, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azure/cli/__main__.py", line 49, in <module>
    exit_code = cli_main(az_cli, sys.argv[1:])
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azure/cli/__main__.py", line 36, in cli_main
    return cli.invoke(args)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/knack/cli.py", li

ClientAuthenticationError: --- Logging error ---
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/logging/__init__.py", line 1089, in emit
    self.flush()
  File "/anaconda/envs/azureml_py38/lib/python3.8/logging/__init__.py", line 1069, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/anaconda/envs/azureml_py38/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azure/cli/__main__.py", line 49, in <module>
    exit_code = cli_main(az_cli, sys.argv[1:])
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azure/cli/__main__.py", line 36, in cli_main
    return cli.invoke(args)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/knack/cli.py", line 233, in

In [ ]:
from azure.ai.ml.entities import Model
# Find the best model

if returned_sweep_job.status == "Completed":

    # First let us get the run which gave us the best result
    best_run = returned_sweep_job.properties["best_child_run_id"]

    # lets get the model from this run
    model = Model(
        # the script stores the model as "outputs"
        path="azureml://jobs/{}/outputs/artifacts/paths/outputs/".format(best_run),
        name="run-model-example",
        description="Model created from run.",
        type="custom_model",
    )

else:
    print(
        "Sweep job status: {}. Please wait until it completes".format(
            returned_sweep_job.status
        )
    )

In [ ]:
# Create an endpoint to deploy your model

import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "serengeti-images-endpoint-" + str(uuid.uuid4())[:8]

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Classify Serengei animals",
    auth_mode="key",
    tags={"data": "snapshot-serengeti", "method": "transfer learning", "framework": "pytorch"},
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

In [ ]:
# Retrieve the endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

In [ ]:
#